<a href="https://colab.research.google.com/github/tingleica/FundingAnalysis/blob/main/ChatGPTCodeDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To install yfinance in Google Colab, you can use the following command:

!pip install yfinance
You can run this command in a code cell in your Colab notebook.

Here's an example of how you can install yfinance and use it to download historical prices of a stock in Colab:



In [ ]:
!pip install yfinance

import yfinance as yf


# Download data and calculate returns

write python code for google colab to download historical prices for a list of stocks saved in a csv file, the calculate the daily returns, and save all of the data and results in a excel file in a local drive

In [ ]:
import pandas as pd
import yfinance as yf

# Load the list of stocks from the CSV file
stocks_csv_path = "/content/SP500List.csv"  # Update with the path to your CSV file
stocks_df = pd.read_csv(stocks_csv_path)

# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Create an empty DataFrame to store the daily returns
returns_data = pd.DataFrame()
# Download historical prices for each stock
start_date = "2013-01-01"
end_date = "2023-07-06"

for TICKER in stocks_df['TICKER']:
    stock = yf.download(TICKER, start=start_date, end=end_date, interval='1mo')
    stock_data = stock[['Adj Close']].rename(columns={'Adj Close': TICKER})
    combined_data = pd.concat([combined_data, stock_data], axis=1)
    #returns_data[TICKER] = stock_data.pct_change().iloc[:, 0]
    returns_data = pd.concat([returns_data, stock_data.pct_change().iloc[:, 0]], axis=1)
# Save the combined data to an Excel file
excel_file_path = "/content/stock_data.xlsx"  # Update with the desired path for the Excel file

with pd.ExcelWriter(excel_file_path) as writer:
    combined_data.to_excel(writer, sheet_name='Combined Data', index_label='Date')

print("Combined data has been saved to:", excel_file_path)

# Save the daily returns to a CSV file
returns_csv_path = "/content/returns_data.csv"  # Update with the desired path for the CSV file

returns_data.to_csv(returns_csv_path, index_label='Date')

print("Returns have been saved to:", returns_csv_path)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: ignored

# Regression between each stock in the SP500 list and SPX

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the daily returns from the CSV file
returns_csv_path = "/content/returns_data.csv"  # Update with the path to your CSV file
returns_data = pd.read_csv(returns_csv_path, index_col=0)

# Perform regression for each TICKER with SPY as the independent variable
regression_results = pd.DataFrame(columns=["TICKER", "Slope", "Intercept", "R-squared"])

# Select SPY as the independent variable
x = returns_data['SPY']

for TICKER in returns_data.columns:
    if TICKER != 'SPY':
        y = returns_data[TICKER]

        # Remove missing Y values
        data = pd.concat([x, y], axis=1).dropna()
        x_cleaned = data['SPY']
        y_cleaned = data[TICKER]

        # Add constant term to independent variable
        x_cleaned = sm.add_constant(x_cleaned)

        # Fit linear regression model
        model = sm.OLS(y_cleaned, x_cleaned)
        results = model.fit()

        # Store regression results
        slope = results.params[1]
        intercept = results.params[0]
        r_squared = results.rsquared
        regression_results = regression_results.append({"TICKER": TICKER, "Slope": slope, "Intercept": intercept, "R-squared": r_squared}, ignore_index=True)

# Save the regression results to a CSV file
regression_csv_path = "/content/regression_results.csv"  # Update with the desired path for the CSV file
regression_results.to_csv(regression_csv_path, index=False)

print("Regression results have been saved to:", regression_csv_path)


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the daily returns from the CSV file
returns_csv_path = "/content/returns_data.csv"  # Update with the path to your CSV file
returns_data = pd.read_csv(returns_csv_path, index_col=0)

# Perform regression for each TICKER with SPY as the independent variable
regression_results = pd.DataFrame(columns=["TICKER", "Slope", "Intercept", "R-squared"])

# Select SPY as the independent variable
x = returns_data['SPY']

# Create an empty DataFrame to store the predicted Y values and errors
predicted_values = pd.DataFrame(index=x.index)
errors = pd.DataFrame(index=x.index)

for TICKER in returns_data.columns:
    if TICKER != 'SPY':
        y = returns_data[TICKER]

        # Remove missing Y values
        data = pd.concat([x, y], axis=1).dropna()
        x_cleaned = data['SPY']
        y_cleaned = data[TICKER]

        # Add constant term to independent variable
        x_cleaned = sm.add_constant(x_cleaned)

        # Fit linear regression model
        model = sm.OLS(y_cleaned, x_cleaned)
        results = model.fit()

        # Store regression results
        slope = results.params[1]
        intercept = results.params[0]
        r_squared = results.rsquared
        regression_results = regression_results.append({"TICKER": TICKER, "Slope": slope, "Intercept": intercept, "R-squared": r_squared}, ignore_index=True)

        # Calculate predicted Y values
        predicted_y = results.predict(x_cleaned)
        predicted_values[TICKER] = predicted_y

        # Calculate error (difference between actual Y and predicted Y)
        error = y_cleaned - predicted_y
        errors[TICKER] = error

# Save the regression results to a CSV file
regression_csv_path = "/content/regression_results.csv"  # Update with the desired path for the CSV file
regression_results.to_csv(regression_csv_path, index=False)

print("Regression results have been saved to:", regression_csv_path)

# Save the predicted Y values to a CSV file
predicted_csv_path = "/content/predicted_values.csv"  # Update with the desired path for the CSV file
predicted_values.to_csv(predicted_csv_path)

print("Predicted Y values have been saved to:", predicted_csv_path)

# Save the errors to a CSV file
errors_csv_path = "/content/errors.csv"  # Update with the desired path for the CSV file
errors.to_csv(errors_csv_path)

print("Errors have been saved to:", errors_csv_path)


Calculate correlation matrix for error terms

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the daily returns from the CSV file
returns_csv_path = "/content/returns_data.csv"  # Update with the path to your CSV file
returns_data = pd.read_csv(returns_csv_path, index_col=0)

# Perform regression for each TICKER with SPY as the independent variable
regression_results = pd.DataFrame(columns=["TICKER", "Slope", "Intercept", "R-squared"])

# Select SPY as the independent variable
x = returns_data['SPY']

# Create an empty DataFrame to store the predicted Y values and errors
predicted_values = pd.DataFrame(index=x.index)
errors = pd.DataFrame(index=x.index)

for TICKER in returns_data.columns:
    if TICKER != 'SPY':
        y = returns_data[TICKER]

        # Remove missing Y values
        data = pd.concat([x, y], axis=1).dropna()
        x_cleaned = data['SPY']
        y_cleaned = data[TICKER]

        # Add constant term to independent variable
        x_cleaned = sm.add_constant(x_cleaned)

        # Fit linear regression model
        model = sm.OLS(y_cleaned, x_cleaned)
        results = model.fit()

        # Store regression results
        slope = results.params[1]
        intercept = results.params[0]
        r_squared = results.rsquared
        regression_results = regression_results.append({"TICKER": TICKER, "Slope": slope, "Intercept": intercept, "R-squared": r_squared}, ignore_index=True)

        # Calculate predicted Y values
        predicted_y = results.predict(x_cleaned)
        predicted_values[TICKER] = predicted_y

        # Calculate error (difference between actual Y and predicted Y)
        error = y_cleaned - predicted_y
        errors[TICKER] = error

# Calculate the correlation matrix for the errors
correlation_matrix = errors.corr(method='pearson')

# Save the regression results to a CSV file
regression_csv_path = "/content/regression_results.csv"  # Update with the desired path for the CSV file
regression_results.to_csv(regression_csv_path, index=False)

print("Regression results have been saved to:", regression_csv_path)

# Save the predicted Y values to a CSV file
predicted_csv_path = "/content/predicted_values.csv"  # Update with the desired path for the CSV file
predicted_values.to_csv(predicted_csv_path)

print("Predicted Y values have been saved to:", predicted_csv_path)

# Save the errors to a CSV file
errors_csv_path = "/content/errors.csv"  # Update with the desired path for the CSV file
errors.to_csv(errors_csv_path)

print("Errors have been saved to:", errors_csv_path)

# Save the correlation matrix to a CSV file
correlation_csv_path = "/content/correlation_matrix.csv"  # Update with the desired path for the CSV file
correlation_matrix.to_csv(correlation_csv_path)

print("Correlation matrix has been saved to:", correlation_csv_path)
